# Chapter 3. Dictionaries and Sets
---

## ToC

1. [Pattern Matching with Mappings](#pattern-matching-with-mappings)
2. [Standard API of Mapping Type](#standard-api-of-mapping-type)  
    2.1. [What is hashable](#what-is-hashable)  
    2.2. [Overview of common mapping methods](#overview-of-common-mapping-methods)  
    2.3. [Inserting or updating mutable values](#inserting-or-updating-mutable-values)
---

## Pattern Matching with Mappings

The `match/case` statement supports subjects that are mapping objects. Patterns for
mappings look like `dict` literals, but they can match instances of any actual or virtual
subclass of `collections.abc.Mapping`.

In [41]:
def get_creators(record: dict) -> list:
    match record:
        case {'type': 'book', 'api': 2, 'authors': [*names]}:
            return names
        case {'type': 'book', 'api': 1, 'author': name}:
            return [name]
        case {'type': 'book', 'api': 3, 'author': name}:
            return name.split()
        case {'type': 'book'}:
            raise ValueError(f"Invalid 'book' record: {record!r}")
        case {'type': 'movie', 'director': name}:
            return [name]
        case _:
            raise ValueError(f'Invalid record: {record!r}')

In [35]:
b1 = dict(
        api=1,
        author='Douglas Hofstadter',
        type='book',
        title='Gödel, Escher, Bach'
    )
b1    

{'api': 1,
 'author': 'Douglas Hofstadter',
 'type': 'book',
 'title': 'Gödel, Escher, Bach'}

In [36]:
get_creators(b1)

['Douglas Hofstadter']

In [37]:
b2 = dict(
        api=2, 
        type='book',
        title='Python in a Nutshell',
        authors='Martelli, Ravenscroft, Holden'.split(sep = ',')
    )
b2

{'api': 2,
 'type': 'book',
 'title': 'Python in a Nutshell',
 'authors': ['Martelli', ' Ravenscroft', ' Holden']}

In [38]:
get_creators(b2)

['Martelli', ' Ravenscroft', ' Holden']

In [42]:
b3 = dict(
        api=3, 
        type='book',
        title='DummyBook',
        author='Author1 Author2 Author3'
    )
b3

{'api': 3,
 'type': 'book',
 'title': 'DummyBook',
 'author': 'Author1 Author2 Author3'}

In [43]:
get_creators(b3)

['Author1', 'Author2', 'Author3']

In [44]:
get_creators({'type': 'book', 'pages': 770})

ValueError: Invalid 'book' record: {'type': 'book', 'pages': 770}

In [45]:
get_creators('Spam, spam, spam')

ValueError: Invalid record: 'Spam, spam, spam'

In contrast with sequence patterns, mapping patterns succeed on partial matches. In the doctests, the b1 and b2 subjects include a 'title' key that does not appear in any 'book' pattern, yet they match.

In [1]:
def match_anything(obj):
    match obj:
        case {'type': 'book', 'author': name}:
            return f"Book by {name}"
        case [first, *rest]:
            return f"Sequence starting with {first}"
        case _:
            return "Other"

print(match_anything({'type': 'book', 'author': 'Orwell', 'title': '1984'}))
# Matches: extra 'title' is fine

print(match_anything(['a', 'b', 'c']))
# Matches because of unpacking with *rest

print(match_anything(['a', 'b', 'c', 'd']))
# Still matches: [fi
# rst, *rest] can handle arbitrary length

print(match_anything(['a']))
# Matches too: rest = []

Book by Orwell
Sequence starting with a
Sequence starting with a
Sequence starting with a


There is no need to use `**extra` to match extra key-value pairs, but if you want to capture them as a dict, you can prefix one variable with `**`. It must be the last in the pattern, and `**_` is forbidden because it would be redundant

In [11]:
food = dict(category='ice cream', flavor='vanilla', cost=199)
match food:
    case {'category': 'ice cream', **details}:
        print(f'Ice cream details: {details}')

Ice cream details: {'flavor': 'vanilla', 'cost': 199}


![Figure 34](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/34.PNG)

## Standard API of Mapping Type

The `collections.abc` module provides the `Mapping` and `MutableMapping` ABCs describing the interfaces of dict and similar types.

The main value of the ABCs is documenting and formalizing the standard interfaces
for mappings, and serving as criteria for isinstance tests in code that needs to support
mappings in a broad sense:

In [15]:
from collections import abc
my_dict = {}
isinstance(my_dict, abc.Mapping)

True

In [16]:
isinstance(my_dict, abc.MutableMapping)

True

![Figure 35](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/35.PNG)

To implement a custom mapping, it’s easier to extend `collections.UserDict`, or to
wrap a `dict` by composition, instead of subclassing these ABCs. The `collec
tions.UserDict` class and all concrete mapping classes in the standard library encapsulate
the basic `dict` in their implementation, which in turn is built on a hash table.
Therefore, they all share the limitation that the keys must be *hashable* (the values
need not be hashable, only the keys).

### What is hashable?

Here is part of the definition of hashable adapted from the [Python Glossary](https://fpy.li/3-3):

> An object is hashable if it has a hash code which never changes during its lifetime (it
needs a `__hash__()` method), and can be compared to other objects (it needs an
`__eq__()` method). Hashable objects which compare equal must have the same hash
code.

Numeric types and flat immutable types `str` and `bytes` are all hashable.
Container types are hashable if they are immutable and all contained objects are also hashable.
A `frozenset` is always hashable, because every element it contains must be hashable by def.
A `tuple` is hashable only if all its items are hashable.

User-defined types are hashable by default because their hash code is their `id()`, and the `__eq__()` method inherited from the object class simply compares the object IDs.

In [21]:
tt = (1, 2, (30, 40))
hash(tt)

-3907003130834322577

In [25]:
tl = (1, 2, [30, 40])
hash(tl)

TypeError: unhashable type: 'list'

In [26]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

5149391500123939311

### Overview of Common Mapping Methods

![Figure 36](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/36.PNG)

The way `d.update(m)` handles its first argument m is a prime example of *duck typing*:
it first checks whether m has a keys method and, if it does, assumes it is a mapping.
Otherwise, `update()` falls back to iterating over m, assuming its items are (key,
value) pairs. The constructor for most Python mappings uses the logic of `update()`
internally, which means they can be initialized from other mappings or from any
iterable object producing (key, value) pairs. A subtle mapping method is `setdefault()`. It avoids redundant key lookups when
we need to update the value of an item in place.

### Inserting or Updating Mutable Values

In line with Python’s fail-fast philosophy, dict access with `d[k]` raises an error when `k` is not an existing key.
`d.get(k, default)` is an alternative to `d[k]` whenever a default value is more convenient than handling KeyError.

**Example:** Build an index mapping word -> list of occurrences  
Consider a script to index text, producing a mapping where each key is a word, and the value is a list of positions where that word occurs:

In [31]:
import this
import codecs

with open("zen.txt", "w", encoding="utf-8") as f:
    decoded_zen = codecs.decode(this.s, "rot_13")
    f.write(decoded_zen)

In [ ]:
import re

WORD_RE = re.compile(r'\w+')
index = {}

# for terminal 
# with open(sys.argv[1], encoding='utf-8') as fp:
# for notebook
with open("zen.txt", encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start() + 1
            location = (line_no, column_no)
            # this is ugly; coded like this to make a point
            # Get the list of occurrences for word, or [] if not found
            occurrences = index.get(word, [])
            # Append new location to occurrences.
            occurrences.append(location)
            # Put changed occurrences into index dict; this entails a second search throught the index
            index[word] = occurrences

# the sorted function  uses the key str.upper to normalize the words for sorting
for word in sorted(index, key=str.upper):
    print(word, index[word])


a [(19, 48), (20, 53)]
Although [(11, 1), (16, 1), (18, 1)]
ambiguity [(14, 16)]
and [(15, 23)]
are [(21, 12)]
aren [(10, 15)]
at [(16, 38)]
bad [(19, 50)]
be [(15, 14), (16, 27), (20, 50)]
beats [(11, 23)]
Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9), (8, 11), (17, 8), (18, 25)]
break [(10, 40)]
by [(1, 20)]
cases [(10, 9)]
complex [(5, 23)]
Complex [(6, 1)]
complicated [(6, 24)]
counts [(9, 13)]
dense [(8, 23)]
do [(15, 64), (21, 48)]
Dutch [(16, 61)]
easy [(20, 26)]
enough [(10, 30)]
Errors [(12, 1)]
explain [(19, 34), (20, 34)]
Explicit [(4, 1)]
explicitly [(13, 8)]
face [(14, 8)]
first [(16, 41)]
Flat [(7, 1)]
good [(20, 55)]
great [(21, 28)]
guess [(14, 52)]
hard [(19, 26)]
honking [(21, 20)]
idea [(19, 54), (20, 60), (21, 34)]
If [(19, 1), (20, 1)]
implementation [(19, 8), (20, 8)]
implicit [(4, 25)]
In [(14, 1)]
is [(3, 11), (4, 10), (5, 8), (6, 9), (7, 6), (8, 8), (17, 5), (18, 16), (19, 23), (20, 23)]
it [(15, 67), (19, 43), (20, 43)]
let [(21, 42)]
m

This was suboptimal script written to show one case where dict.get is not the best way to handle a missing key. The three lines dealing with occurrences in the Example can be replaced by a single line using `dict.setdefault`. Version below is closer to Alex Martelli’s code.

In [35]:
import re
import sys

WORD_RE = re.compile(r'\w+')
index = {}

# for terminal 
# with open(sys.argv[1], encoding='utf-8') as fp:
# for notebook
with open("zen.txt", encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start() + 1
            location = (line_no, column_no)
            index.setdefault(word, []).append(location)

# display in alphabetical order
for word in sorted(index, key=str.upper):
    print(word, index[word])    

a [(19, 48), (20, 53)]
Although [(11, 1), (16, 1), (18, 1)]
ambiguity [(14, 16)]
and [(15, 23)]
are [(21, 12)]
aren [(10, 15)]
at [(16, 38)]
bad [(19, 50)]
be [(15, 14), (16, 27), (20, 50)]
beats [(11, 23)]
Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9), (8, 11), (17, 8), (18, 25)]
break [(10, 40)]
by [(1, 20)]
cases [(10, 9)]
complex [(5, 23)]
Complex [(6, 1)]
complicated [(6, 24)]
counts [(9, 13)]
dense [(8, 23)]
do [(15, 64), (21, 48)]
Dutch [(16, 61)]
easy [(20, 26)]
enough [(10, 30)]
Errors [(12, 1)]
explain [(19, 34), (20, 34)]
Explicit [(4, 1)]
explicitly [(13, 8)]
face [(14, 8)]
first [(16, 41)]
Flat [(7, 1)]
good [(20, 55)]
great [(21, 28)]
guess [(14, 52)]
hard [(19, 26)]
honking [(21, 20)]
idea [(19, 54), (20, 60), (21, 34)]
If [(19, 1), (20, 1)]
implementation [(19, 8), (20, 8)]
implicit [(4, 25)]
In [(14, 1)]
is [(3, 11), (4, 10), (5, 8), (6, 9), (7, 6), (8, 8), (17, 5), (18, 16), (19, 23), (20, 23)]
it [(15, 67), (19, 43), (20, 43)]
let [(21, 42)]
m

the end result of this line…
```python
my_dict.setdefault(key, []).append(new_value)
```
…is the same as running…
```python
if key not in my_dict:
my_dict[key] = []
my_dict[key].append(new_value)
```
…Except that the latter code performs at least two searches for key—three if it’s not
found—while setdefault does it all with a single lookup.

The section of next notebook (handling missing keys on any lookup, and not only when inserting) is realted to current subject.